### PERTUKARAN KUNCI DIFFIE HELLMAN

Kalau dua pihak saling berbagi kunci publik masing-masing, maka dengan metoda DH ini dapat dihasilkan suatu kunci rahasia yang dapat digunakan dalam enkripsi simetris untuk mengirim pesan rahasia selanjutnya.    
Enkripsi asimetris melibatkan dua kuci yaitu kunci pribadi yang dirahasiakan dan kunci publik yang diumumkan atau dikirimkan secara terbuka, dan menggunakan perhitungan matematis yang berat dibandingkan perhitungan enkripsi simetris yang menggunakan satu kunci rahasia bersama (simetris).    
Makanya enkripsi asimetris biasanya digunakan untuk 'berbagi' kunci simetris lalu pesan-pesan selanjutnya dienkripsi secara simetris dengan kunci bersama itu.    

Metoda pertukaran kunci metoda Diffie Hellman (1976) dapat dijelaskan secara sederhana menggunakan prinsip bahwa
> (g<sup>a</sup>)<sup>b</sup> = (g<sup>b</sup>)<sup>a</sup> = g<sup>ab</sup>

Si Ani merahasiakan pin a tetapi membagi nilai g<sup>a</sup>   
Si Budi merahasiakan pin b tetapi membagi nilai g<sup>b</sup>   
Penyadap yang mengetahui g<sup>a</sup> dan g<sup>b</sup> tidak bisa menghitung g<sup>ab</sup>; mengalikan keduanya hanya menghasilkan g<sup>a+b</sup>   
Generator g tidak dirahasiakan nilainya biasanya 2 atau 3 atau bilangan prima kecil. Agar menyulitkan penyadap menghitung a dari g<sup>a</sup> disertakanlah *problema logaritme-diskrit* yaitu perpangkatan modulus sehingga kunci publiknya menjadi sisa g<sup>a</sup> dibagi p, atau (g<sup>a</sup>)modulo p. Bilangan p adalah angka prima yang besar misalnya 2048 bit, atau 8192 bit, dan semua operasi itu tadi berada dalam medan modulo p, yaitu hasil perhitungan dijamin tidak lebih besar dari p. Bilangan p (atau dsebut juga modp) tidak rahasia, dapat dilihat dari usulan RFC3526 di internet.   
Oleh karena itu pin a jangan berupa angka yang kecil, supaya jika penyadap mencoba coba urut kacang untuk menghasilkan kunci publik yang cocok, bakal memakan waktu yang lama; disarankan pin a lebih dari 600 digit angka dengan menggunakan suatu metoda khusus yang dirahasiakan. Passphrase yang hanya 20~50 karakter dapat dibuat menjadi pin 1000 lebih digit angka, hanya metodanya harus dirahasiakan, kalau tidak penyadap akan menebak/mencoba (brute force) passphrasenya.      


In [8]:
# perhitungan kunci publik melibatkan angka yang luar biasa besar
# gunakan built-in library pow() untuk menghitung
# contoh:
a = 65537 # hanya 5 digit angka
g = 2
# bagian kunci publik tanpa pembagi modulo
kp = pow(g,a) # g**a memberikan hasil yang sama kalau komputer Anda tidak keburu hang
print(f'jumlah digit: {len(str(kp))}') # jauh lebih besar dari modp
print(kp)  # sekedar ingin tahu saja

jumlah digit: 19729
40070598608136929299581447031205115008956509511395028385300339474217881191126229061790122617618666962020764686858145263636458987642376253377390127295230940583300837438327031759326944388858618559641686182097119811403186379192790497267447344060058339391843122175298977785081816118229140753504170004133431274047322527194942896142235496317618282714854419343803036725651212361829177053996522828500602467822165472076875357528980864119207582489818114151206280701523251249520637275862529694074875659099512275419632092288266173842362049719183047603906620605843256003211373402113032935011360774830589276844896905850747228850672287474581766075892025494499168297298318612945040303113878452563613833015927621282645506145342879963170176225852578022684755654111348421601401305679266443101556624285771033511081466902144262248547991259654395383001097678104476087140916963959127863157070200379840000482839274136271196809280789443880321390353803122394539646757800352830343801022669326137962804387669

Jadi Ani menyimpan nilai a dan menghitung (g<sup>a</sup>)modulo p untuk Budi, menerima nilai (g<sup>b</sup>)modulo p dari Budi, lalu menghitung
(g<sup>b</sup>)<sup>a</sup> modulo p = g<sup>ab</sup> modulo p   
Demikian juga Budi menghitung (g<sup>a</sup>)<sup>b</sup> modulo p = g<sup>ab</sup> modulo p yang digunakan sebagai kunci rahasia bersama, kunci simetris.


In [1]:
# dh8192.py
import random

# modp 8192-bit RFC3526
p8192_str = \
'FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD1'\
'29024E088A67CC74020BBEA63B139B22514A08798E3404DD'\
'EF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245'\
'E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7ED'\
'EE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3D'\
'C2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F'\
'83655D23DCA3AD961C62F356208552BB9ED529077096966D'\
'670C354E4ABC9804F1746C08CA18217C32905E462E36CE3B'\
'E39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9'\
'DE2BCBF6955817183995497CEA956AE515D2261898FA0510'\
'15728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64'\
'ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7'\
'ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6B'\
'F12FFA06D98A0864D87602733EC86A64521F2B18177B200C'\
'BBE117577A615D6C770988C0BAD946E208E24FA074E5AB31'\
'43DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D7'\
'88719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA'\
'2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6'\
'287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED'\
'1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA9'\
'93B4EA988D8FDDC186FFB7DC90A6C08F4DF435C934028492'\
'36C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BD'\
'F8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831'\
'179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1B'\
'DB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF'\
'5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6'\
'D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F3'\
'23A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AA'\
'CC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE328'\
'06A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55C'\
'DA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE'\
'12BF2D5B0B7474D6E694F91E6DBE115974A3926F12FEE5E4'\
'38777CB6A932DF8CD8BEC4D073B931BA3BC832B68D9DD300'\
'741FA7BF8AFC47ED2576F6936BA424663AAB639C5AE4F568'\
'3423B4742BF1C978238F16CBE39D652DE3FDB8BEFC848AD9'\
'22222E04A4037C0713EB57A81A23F0C73473FC646CEA306B'\
'4BCBC8862F8385DDFA9D4B7FA2C087E879683303ED5BDD3A'\
'062B3CF5B3A278A66D2A13F83F44F82DDF310EE074AB6A36'\
'4597E899A0255DC164F31CC50846851DF9AB48195DED7EA1'\
'B1D510BD7EE74D73FAF36BC31ECFA268359046F4EB879F92'\
'4009438B481C6CD7889A002ED5EE382BC9190DA6FC026E47'\
'9558E4475677E9AA9E3050E2765694DFC81F56E880B96E71'\
'60C980DD98EDD3DFFFFFFFFFFFFFFFFF'
modp = int(p8192_str,16)
#print(len(str(modp)))
g = 2

# alice's part
private_a = 12345678901223334444555556666667777777888888889999999990000000000
public_a = pow(g,private_a,modp)  # fungsi pow() jauh lebih cepat daripada polosan (g**a)%p
print(f'length of Alice\' public key {len(str(public_a))} digits')

# bob's part
private_b = random.randint(65537,99999)
public_b = pow(g, private_b, modp)
print(f'length of Bob\' public key {len(str(public_b))} digits')

# exchange public keys
# each part then generate the same symetric key
print('both parties calculate the simetric key')
simkey_a = pow(public_b, private_a, modp)
simkey_b = pow(public_a, private_b, modp)
print(f'are they the same? {simkey_b == simkey_a = }\n')
print(f'simetric secretkey {len(str(simkey_a))} digits :\n{simkey_b}')

length of Alice' public key 2466 digits
length of Bob' public key 2465 digits
both parties calculate the simetric key
are they the same? simkey_b == simkey_a = True

simetric secretkey 2466 digits :
470005951707455628987984370962562128099318456419865530953722672317112464586907687081966098127101024389743262211880115357159976681237373684655973199013947375647121900032974494825069134333588543883522312835941725103173200945182723433402902798394117578249484051102507460292250346358828305252115505840427493985891513990865078078808187951046266369085560297448116655877900437907905270962492133057902257446606459378298237035946907856342376073772309273786349805608856215609104841106870761135709159503485812089934198150607785383319463779877918389318562736849369390717728092897820368528367738540355229504069061906422628463697198046604259586094247644381654487864060294867063919906282515342561829145196133169225673677754295890347667102971611721564405106873210619810962497299356736882928902458151029436798851131125

Untuk mengkomunikasikan kunci publik secara lebih ringkas dapat kita gunakan pengkodean dari angka menjadi string alfanumerik.

In [2]:
# Mengkodekan pin menjadi password
alnum94 = "".join([chr(i) for i in range(33,127)])

def enc94(number:int) -> str:
    number =  int(number)
    if number == 0:
        return '0'
    kode = []
    while number != 0:
        sisa = number %94
        kode.append(alnum94[sisa])
        number = number // 94
    return ("".join(reversed(kode)))  # kode string output tidak pakai spasi.
    
def dec94(kode:str) -> int:  # kode input tanpa spasi
    def digpow(t):
        return alnum94.find(t[1]) * pow(94,t[0])
    return sum(map(digpow,enumerate(kode[-1::-1])))

kode_public_Ani = enc94(public_a)
kode_public_Budi = enc94(public_b)
print(f'kode publik Ani {len(kode_public_Ani)} karakter:\n{kode_public_Ani}')
print()
print(f'kode publik Budi {len(kode_public_Budi)} karakter:\n{kode_public_Budi}')

kode publik Ani 1250 karakter:
CF-Gm#y-PSvgl>*JxIn\/.+jO,*k|dwNb5)*a~tAms&a^}[T(i#K2L2yK%-u&D9GYIgEu;5hbB17qk3e:j$BuG]F8+o.C*2PZvAE~31"?\{2pPrDbInP2a?vz'Qx3[=T+dLA|j%f;*vLRnZd-N`km'k*O/pV+(7eiD{)v9TWM.h,".I=CQCkG"o_0G8reJ5>/4,QsIi?40L&jE;`mAMkN$30~!&|/&j~4{vPqGd$14y*5tP_UXa3t2|eO?tisO\{|hm<Y{YNrN)Ep~:L<vf$orGQ:7TA-a#],%@/&/Kd=<$j~_g)8ka+{*^@eEFgjCIX8a:>BA]g9d.nLX\:)%Ar}hAc3HyWvA2#)bM;=rL:!tJ4J&b!5@er;jck7rLi-[eq8@jlb=H$n=ApGS[zMi5Gv"!>+<s6%I#~O1q,}2}^nRJd*LnFcxR+fO'IKD_4|d]~[C`(Y:n,$"a0[97c#|4fHwd(RwMjtyeLc*RzX60fG^VJ\)+ZxF/+y=XNw1PmAcgD+;GjcO:o!3\ij/a$mji]U/g>mnF?H;3OU5+'`&4Xkp{YEd|?V@/O;)_.[}@Yh~-2)&E@5#H1$g[=g/S_;u$%H$?;1a*TK1f)?HXJuJDw"p<#%O2&/mP+:Yn>{7},5d/IQC^>EaW]-CDjORq6*A'vuNbUnrvn$&x7ay"F,JkmH"'c*Da<4KGS~`kj>g.T$KVt>}SHm)$\-s5;o3lx]6"|wQynx\s0;v8F.oE~fU@9cP>,Hsv~-,#Yfv|v!q015mN%T2,K"ucw!EF\WuHTOoF,2<b$cu{3D"zeb\/S(u5bX!$';iHYdhJ~EthGLisDAhK28gOjZ1Or6Gh0IP<gF)R}-:|9=XeKpO2,'m*>@)nuw'n5]YECu*jrRNI&Hbr>u0<Ohd,bc-%b@iAlWB#{mXbO@0.-mD8V]O.)<d5vBImo<"tK[9o6gSo=*LUG~phRkJCj"p`M;iUZp

Demikian guys pembahasan mengenai pertukaran kunci Diffie Hellman dengan catatan bahwa kunci pribadi/privat harus cukup lumayan panjang (setidaknya 600 angka) supaya lama jika dicoba-coba (brute force). Sebaiknya juga menggunakan kode password atau passphrase yang kemudian didekode menjadi deretan angka, atau juga menggunakan **formula tertentu yang dirahasiakan** seperti pada pembahasan yang lalu dengan menggunakan hash blake2b.